In [ ]:
def find_good_trials(df, fs, criteria_set, probe_vel,
                       trial_type='', series_type=''):
    """
    Trials are useful based on 4 conditions:
    1) State-evoking epoch started
    2) Probe was successfully delivered
    3) Fish swam to a certain degree during the EVOKE epoch
    4) Fish stopped swimming when it either gives up or stimulus is withheld
    Parameters
    ----------
    df : dataframe containing all trials
    fs : aquisition rate of epoch_series, trial_indices, probe_indices
    criteria_set : predefined sets of criteria for checking.
    trial_type : was originally 'OL' and 'CL', but as I expand my 
                 experimental paradigm, could be 'HG', 'LG' and 'NG'
    series_type : used to indicate whether the series was downsampled
                  in time; has to do with fs 
    Returns
    -------
    useful_df : returns a dataframe containing only the useful trials
    """
    ## if series_type is provided, append an underscore to it
    if series_type: series_type = series_type+'_'
    if criteria_set == 1:  ## Check criteria for Open loop trial
        ## I can't really control for how long the fish swims and
        # struggles before giving up, so a minimum swim time doesn't
        # work too well here... All I'm check is that the fish starts
        # swimming a bit before going into open loop (resetting state
        # to baseline) and then also experiences open loop by
        # swimming in the evoke epoch.
        trial_params = {'swim before EVOKE': -1,  # default = -1
                        'swim after EVOKE': 2,  # default = 2
                        'no swim before PROBE': -5}  # default = -5
        df['trial_params'] = [trial_params] * len(df)
        ## check FOR swimming 2 sec before state-evoking epoch
        trigger = ['evoke start_idx', fs]
        around = [trial_params['swim before EVOKE'],0]
        check_pre_evoke_swimming = partial(check_swimming, trigger=trigger, around=around)
        df['pre-evoke_swimming'] = df.apply(check_pre_evoke_swimming,axis=1)
        ## check FOR swimming 1 sec after the onset of state-evoking epoch
        trigger = ['evoke start_idx', fs]
        around = [0,trial_params['swim after EVOKE']]
        check_post_evoke_swimming = partial(check_swimming, trigger=trigger, around=around)
        df['post-evoke_swimming'] = df.apply(check_post_evoke_swimming,axis=1)
        ## check FOR *NO* swimming 5 seconds before going into the
        # probe epoch, presumably because the fish has given up
        trigger = ['probe start_idx', fs]
        around = [trial_params['no swim before PROBE'],-0.1]
        swim = False
        check_pre_probe_swimming = partial(check_swimming, trigger=trigger, around=around, swim=swim)
        df['pre-probe_swimming'] = df.apply(check_pre_probe_swimming,axis=1)        
        useful_df = df.loc[
            (df['trial_label'] == trial_type) &
            (~np.isnan(df[series_type+'evoke start_idx'])) &
            (~np.isnan(df[series_type+'probe start_idx'])) &
            (~np.isnan(df[series_type+'recovery start_idx'])) &
            (df[series_type+'recovery_idx'].map(lambda x: len(x)) > 6000) &
            (df['pre-evoke_swimming']==True) &
            (df['post-evoke_swimming']==True) &
            (df['pre-probe_swimming']==True) &  ## essentially False            
            (df['probevel']==probe_vel) 
            # (~np.isnan(df[series_type+trial_type+'REC_idx']))
        ]
    elif criteria_set == 2:  ## Check criteria for closed loop trial
                             ## at least 4 second swimming any time during the epoch (about 3 bouts)
        trial_params = {'swim during EVOKE': 4,  ## 4
                        'no swim before PROBE': -5}
        df['trial_params'] = [trial_params] * len(df)        
        ## check FOR swimming for approximately half the time during
        # the high gain evoke period
        trigger = ['pause start_idx', fs]
        around = [-10,0]
        min_dur = trial_params['swim during EVOKE']
        check_pre_pause_swimming = partial(check_swimming, trigger=trigger, around=around, min_dur=min_dur)
        df['pre-pause_swimming'] = df.apply(check_pre_pause_swimming,axis=1)
        ## check FOR *NO* swimming 5 seconds before going into the probe epoch
        trigger = ['probe start_idx', fs]
        around = [trial_params['no swim before PROBE'],-0.1]
        swim = False
        check_pre_probe_swimming = partial(check_swimming, trigger=trigger, around=around, swim=swim)
        df['pre-probe_swimming'] = df.apply(check_pre_probe_swimming,axis=1)
        useful_df = df.loc[
            (df['trial_label'] == trial_type) &
            (~np.isnan(df[series_type+'evoke start_idx'])) &
            (~np.isnan(df[series_type+'probe start_idx'])) &
            (~np.isnan(df[series_type+'recovery start_idx'])) &
            (df[series_type+'recovery_idx'].map(lambda x: len(x)) > 6000) &            
            (df['pre-pause_swimming']==True) &
            (df['pre-probe_swimming']==True) &  ## essentially False
            (df['probevel']==probe_vel) 
            # (~np.isnan(df[series_type+trial_type+'REC_idx']))
        ]
    return useful_df